PWC-Net-small model training (with cyclical learning rate schedule)
=======================================================

In this notebook we:
- Use a small model (no dense or residual connections), 6 level pyramid, uspample level 2 by 4 as the final flow prediction
- Train the PWC-Net-small model on a mix of the `FlyingChairs` and `FlyingThings3DHalfRes` dataset using a Cyclic<sub>short</sub> schedule of our own
- The Cyclic<sub>short</sub> schedule oscillates between `5e-04` and `1e-05` for 200,000 steps

Below, look for `TODO` references and customize this notebook based on your own needs.

## Reference

[2018a]<a name="2018a"></a> Sun et al. 2018. PWC-Net: CNNs for Optical Flow Using Pyramid, Warping, and Cost Volume. [[arXiv]](https://arxiv.org/abs/1709.02371) [[web]](http://research.nvidia.com/publication/2018-02_PWC-Net%3A-CNNs-for) [[PyTorch (Official)]](https://github.com/NVlabs/PWC-Net/tree/master/PyTorch) [[Caffe (Official)]](https://github.com/NVlabs/PWC-Net/tree/master/Caffe)

In [1]:
"""
pwcnet_train.ipynb

PWC-Net model training.

Written by Phil Ferriere

Licensed under the MIT License (see LICENSE for details)

Tensorboard:
    [win] tensorboard --logdir=E:\\repos\\tf-optflow\\tfoptflow\\pwcnet-sm-6-2-cyclic-chairsthingsmix
    [ubu] tensorboard --logdir=/media/EDrive/repos/tf-optflow/tfoptflow/pwcnet-sm-6-2-cyclic-chairsthingsmix
"""
from __future__ import absolute_import, division, print_function
import sys
from copy import deepcopy

from dataset_base import _DEFAULT_DS_TRAIN_OPTIONS
from dataset_flyingchairs import FlyingChairsDataset
from dataset_flyingthings3d import FlyingThings3DHalfResDataset
from dataset_mixer import MixedDataset
from model_pwcnet import ModelPWCNet, _DEFAULT_PWCNET_TRAIN_OPTIONS

## TODO: Set this first!

In [2]:
# TODO: You MUST set dataset_root to the correct path on your machine!
if sys.platform.startswith("win"):
    _DATASET_ROOT = 'E:/datasets/'
else:
    _DATASET_ROOT = '/media/EDrive/datasets/'
_FLYINGCHAIRS_ROOT = _DATASET_ROOT + 'FlyingChairs_release'
_FLYINGTHINGS3DHALFRES_ROOT = _DATASET_ROOT + 'FlyingThings3D_HalfRes'
    
# TODO: You MUST adjust the settings below based on the number of GPU(s) used for training
# Set controller device and devices
# A one-gpu setup would be something like controller='/device:GPU:0' and gpu_devices=['/device:GPU:0']
# Here, we use a dual-GPU setup, as shown below
gpu_devices = ['/device:GPU:0', '/device:GPU:1']
controller = '/device:CPU:0'

# TODO: You MUST adjust this setting below based on the amount of memory on your GPU(s)
# Batch size
batch_size = 8

# Pre-train on `FlyingChairs+FlyingThings3DHalfRes` mix

## Load the dataset

In [3]:
# TODO: You MUST set the batch size based on the capabilities of your GPU(s) 
#  Load train dataset
ds_opts = deepcopy(_DEFAULT_DS_TRAIN_OPTIONS)
ds_opts['in_memory'] = False                          # Too many samples to keep in memory at once, so don't preload them
ds_opts['aug_type'] = 'heavy'                         # Apply all supported augmentations
ds_opts['batch_size'] = batch_size * len(gpu_devices) # Use a multiple of 8; here, 16 for dual-GPU mode (Titan X & 1080 Ti)
ds_opts['crop_preproc'] = (256, 448)                  # Crop to a smaller input size
ds1 = FlyingChairsDataset(mode='train_with_val', ds_root=_FLYINGCHAIRS_ROOT, options=ds_opts)
ds_opts['type'] = 'into_future'
ds2 = FlyingThings3DHalfResDataset(mode='train_with_val', ds_root=_FLYINGTHINGS3DHALFRES_ROOT, options=ds_opts)
ds = MixedDataset(mode='train_with_val', datasets=[ds1, ds2], options=ds_opts)

In [4]:
# Display dataset configuration
ds.print_config()


Dataset Configuration:
  verbose              False
  in_memory            False
  crop_preproc         (256, 448)
  scale_preproc        None
  input_channels       3
  tb_test_imgs         False
  random_seed          1969
  val_split            0.03
  aug_type             heavy
  aug_labels           True
  fliplr               0.5
  flipud               0.5
  translate            (0.5, 0.05)
  scale                (0.5, 0.05)
  batch_size           16
  type                 into_future
  mode                 train_with_val
  train size           41282
  val size             1230


## Configure the training

In [5]:
# Start from the default options
nn_opts = deepcopy(_DEFAULT_PWCNET_TRAIN_OPTIONS)
nn_opts['verbose'] = True
nn_opts['ckpt_dir'] = './pwcnet-sm-6-2-cyclic-chairsthingsmix/'
nn_opts['batch_size'] = ds_opts['batch_size']
nn_opts['x_shape'] = [2, ds_opts['crop_preproc'][0], ds_opts['crop_preproc'][1], 3]
nn_opts['y_shape'] = [ds_opts['crop_preproc'][0], ds_opts['crop_preproc'][1], 2]
nn_opts['use_tf_data'] = True # Use tf.data reader
nn_opts['gpu_devices'] = gpu_devices
nn_opts['controller'] = controller

# Use the PWC-Net-small model in quarter-resolution mode
nn_opts['use_dense_cx'] = False
nn_opts['use_res_cx'] = False
nn_opts['pyr_lvls'] = 6
nn_opts['flow_pred_lvl'] = 2

In [6]:
# Set the learning rate schedule. This schedule is for a single GPU using a batch size of 8.
# Below,we adjust the schedule to the size of the batch and the number of GPUs.
nn_opts['lr_policy'] = 'cyclic'
nn_opts['cyclic_lr_max'] = 5e-04 # Anything higher will generate NaNs
nn_opts['cyclic_lr_base'] = 1e-05
nn_opts['cyclic_lr_stepsize'] = 20000
nn_opts['max_steps'] = 200000

# Below,we adjust the schedule to the size of the batch and our number of GPUs (2).
nn_opts['max_steps'] = int(nn_opts['max_steps'] * 8 / ds_opts['batch_size'])
nn_opts['cyclic_lr_stepsize'] = int(nn_opts['cyclic_lr_stepsize'] * 8 / ds_opts['batch_size'])

In [7]:
# Instantiate the model and display the model configuration
nn = ModelPWCNet(mode='train_with_val', options=nn_opts, dataset=ds)
nn.print_config()

Building model towers...
  Building tower_0...
Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`
  ...tower_0 built.
  Building tower_1...
  ...tower_1 built.
... model towers built.
Initializing model with random values for initial training...

... model initialized

Model Configuration:
  verbose                True
  ckpt_dir               ./pwcnet-sm-6-2-cyclic-chairsthingsmix/
  max_to_keep            10
  x_dtype                <dtype: 'float32'>
  x_shape                [2, 256, 448, 3]
  y_dtype                <dtype: 'float32'>
  y_shape                [256, 448, 2]
  train_mode             train
  display_step           100
  snapshot_step          1000
  val_step               1000
  val_batch_size         -1
  tb_val_imgs            pyramid
  tb_test_imgs           None
  gpu_devices            ['/device:GPU:0', '/device:GPU:1']
  controller             /device:CPU:0
  use_tf_data            True
  batch_size             16
  lr_policy        

## Train the model

In [8]:
# Train the model
nn.train()

Start training from scratch...
2018-09-08 05:49:28 Iter 100 [Train]: loss=221.19, epe=18.03, lr=0.000020, samples/sec=23.2, sec/step=1.379, eta=19:06:58
2018-09-08 05:50:39 Iter 200 [Train]: loss=217.13, epe=17.71, lr=0.000030, samples/sec=52.1, sec/step=0.614, eta=8:29:35
2018-09-08 05:51:50 Iter 300 [Train]: loss=216.84, epe=17.69, lr=0.000039, samples/sec=52.3, sec/step=0.612, eta=8:27:17
2018-09-08 05:53:01 Iter 400 [Train]: loss=220.07, epe=17.97, lr=0.000049, samples/sec=52.3, sec/step=0.612, eta=8:26:03
2018-09-08 05:54:12 Iter 500 [Train]: loss=217.68, epe=17.78, lr=0.000059, samples/sec=52.3, sec/step=0.612, eta=8:25:12
2018-09-08 05:55:23 Iter 600 [Train]: loss=221.48, epe=18.01, lr=0.000069, samples/sec=51.7, sec/step=0.619, eta=8:29:38
2018-09-08 05:56:35 Iter 700 [Train]: loss=200.21, epe=15.82, lr=0.000079, samples/sec=51.7, sec/step=0.619, eta=8:28:21
2018-09-08 05:57:48 Iter 800 [Train]: loss=193.49, epe=14.99, lr=0.000088, samples/sec=50.6, sec/step=0.632, eta=8:38:29


... model saved in ./pwcnet-sm-6-2-cyclic-chairsthingsmix/pwcnet.ckpt-11000
2018-09-08 08:10:53 Iter 11100 [Train]: loss=86.01, epe=6.29, lr=0.000064, samples/sec=47.8, sec/step=0.669, eta=7:13:39
2018-09-08 08:12:10 Iter 11200 [Train]: loss=85.41, epe=6.26, lr=0.000069, samples/sec=47.3, sec/step=0.677, eta=7:17:46
2018-09-08 08:13:27 Iter 11300 [Train]: loss=81.92, epe=5.98, lr=0.000074, samples/sec=47.3, sec/step=0.677, eta=7:16:43
2018-09-08 08:14:44 Iter 11400 [Train]: loss=88.37, epe=6.49, lr=0.000079, samples/sec=47.4, sec/step=0.675, eta=7:14:20
2018-09-08 08:16:01 Iter 11500 [Train]: loss=83.04, epe=6.07, lr=0.000083, samples/sec=47.4, sec/step=0.675, eta=7:13:19
2018-09-08 08:17:18 Iter 11600 [Train]: loss=86.43, epe=6.34, lr=0.000088, samples/sec=47.2, sec/step=0.678, eta=7:14:12
2018-09-08 08:18:35 Iter 11700 [Train]: loss=83.20, epe=6.10, lr=0.000093, samples/sec=47.0, sec/step=0.681, eta=7:14:37
2018-09-08 08:19:51 Iter 11800 [Train]: loss=85.18, epe=6.23, lr=0.000098, sa

... model saved in ./pwcnet-sm-6-2-cyclic-chairsthingsmix/pwcnet.ckpt-22000
2018-09-08 10:41:36 Iter 22100 [Train]: loss=74.32, epe=5.30, lr=0.000061, samples/sec=44.0, sec/step=0.727, eta=5:38:10
2018-09-08 10:42:59 Iter 22200 [Train]: loss=72.02, epe=5.12, lr=0.000064, samples/sec=43.5, sec/step=0.735, eta=5:40:30
2018-09-08 10:44:22 Iter 22300 [Train]: loss=73.88, epe=5.29, lr=0.000066, samples/sec=43.5, sec/step=0.735, eta=5:39:33
2018-09-08 10:45:44 Iter 22400 [Train]: loss=72.51, epe=5.15, lr=0.000069, samples/sec=43.5, sec/step=0.735, eta=5:38:05
2018-09-08 10:47:07 Iter 22500 [Train]: loss=75.97, epe=5.44, lr=0.000071, samples/sec=43.6, sec/step=0.734, eta=5:36:30
2018-09-08 10:48:30 Iter 22600 [Train]: loss=72.29, epe=5.14, lr=0.000074, samples/sec=43.4, sec/step=0.738, eta=5:36:50
2018-09-08 10:49:53 Iter 22700 [Train]: loss=72.44, epe=5.13, lr=0.000076, samples/sec=43.7, sec/step=0.732, eta=5:32:59
2018-09-08 10:51:16 Iter 22800 [Train]: loss=73.29, epe=5.23, lr=0.000079, sa

... model saved in ./pwcnet-sm-6-2-cyclic-chairsthingsmix/pwcnet.ckpt-33000
2018-09-08 13:24:26 Iter 33100 [Train]: loss=67.33, epe=4.71, lr=0.000048, samples/sec=41.3, sec/step=0.774, eta=3:38:08
2018-09-08 13:25:53 Iter 33200 [Train]: loss=67.43, epe=4.73, lr=0.000049, samples/sec=41.2, sec/step=0.776, eta=3:37:18
2018-09-08 13:27:21 Iter 33300 [Train]: loss=66.81, epe=4.70, lr=0.000050, samples/sec=41.1, sec/step=0.779, eta=3:36:55
2018-09-08 13:28:49 Iter 33400 [Train]: loss=68.18, epe=4.78, lr=0.000052, samples/sec=41.1, sec/step=0.778, eta=3:35:13
2018-09-08 13:30:17 Iter 33500 [Train]: loss=68.39, epe=4.80, lr=0.000053, samples/sec=41.0, sec/step=0.781, eta=3:34:48
2018-09-08 13:31:45 Iter 33600 [Train]: loss=66.06, epe=4.62, lr=0.000054, samples/sec=41.0, sec/step=0.781, eta=3:33:29
2018-09-08 13:33:13 Iter 33700 [Train]: loss=68.08, epe=4.79, lr=0.000055, samples/sec=41.1, sec/step=0.779, eta=3:31:38
2018-09-08 13:34:41 Iter 33800 [Train]: loss=65.89, epe=4.60, lr=0.000057, sa

... model saved in ./pwcnet-sm-6-2-cyclic-chairsthingsmix/pwcnet.ckpt-44000
2018-09-08 16:16:12 Iter 44100 [Train]: loss=68.37, epe=4.80, lr=0.000035, samples/sec=39.2, sec/step=0.815, eta=1:20:11
2018-09-08 16:17:44 Iter 44200 [Train]: loss=64.01, epe=4.44, lr=0.000036, samples/sec=39.3, sec/step=0.815, eta=1:18:48
2018-09-08 16:19:16 Iter 44300 [Train]: loss=66.37, epe=4.64, lr=0.000036, samples/sec=39.1, sec/step=0.818, eta=1:17:45
2018-09-08 16:20:48 Iter 44400 [Train]: loss=66.94, epe=4.66, lr=0.000037, samples/sec=39.4, sec/step=0.811, eta=1:15:44
2018-09-08 16:22:20 Iter 44500 [Train]: loss=65.17, epe=4.54, lr=0.000038, samples/sec=39.1, sec/step=0.817, eta=1:14:56
2018-09-08 16:23:52 Iter 44600 [Train]: loss=65.51, epe=4.57, lr=0.000038, samples/sec=39.0, sec/step=0.821, eta=1:13:53
2018-09-08 16:25:24 Iter 44700 [Train]: loss=64.19, epe=4.44, lr=0.000039, samples/sec=39.3, sec/step=0.814, eta=1:11:54
2018-09-08 16:26:56 Iter 44800 [Train]: loss=67.68, epe=4.73, lr=0.000039, sa

## Training log

Here are the training curves for the run above:

![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix/loss.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix/epe.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix/lr.png)

Here are the predictions issued by the model for a few validation samples:

![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix/val1.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix/val2.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix/val3.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix/val4.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix/val5.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix/val6.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix/val7.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix/val8.png)